In [ ]:
import os
import sys

project_path = os.path.dirname(os.path.abspath(os.getcwd()))
sys.path.append(project_path)
import BraiAn

In [ ]:
# from https://help.brain-map.org/display/api/Downloading+an+Ontology%27s+Structure+Graph
# StructureGraph id=1
path_to_allen_json = os.path.join(project_path, "data", "AllenMouseBrainOntology.json")
branches_to_exclude = ["retina","VS","grv","fiber tracts"]
AllenBrain = BraiAn.AllenBrainHierarchy(path_to_allen_json, branches_to_exclude)

#parent_region = AllenBrain.parent_region
#direct_subregions = AllenBrain.direct_subregions
#full_name = AllenBrain.full_name
#regions = list(full_name.keys())

In [ ]:
# ####################################### SET PARAMETERS ####################################
experiments_root = "../data/experiments/"

experiment_1_dir = "shila"
experiment_1_name = "Shila"
experiment_1_area_key = "Area um^2" #"DAPI: DAPI area um^2"
experiment_1_root = os.path.join(experiments_root, experiment_1_dir)
experiment_1_animals_root = os.path.join(experiment_1_root, "QuPath_output")
experiment_1_dirs = ["16C", "17C", "19C", "5S", "8S", "10S", "13S", "14S", "1R", "2R", "3R", "4R", "11R"]

experiment_2_dir = "soumnya/experiment"
experiment_2_name = "Soumnya"
experiment_2_area_key = "Area um^2"
experiment_2_root = os.path.join(experiments_root, experiment_2_dir)
experiment_2_animals_root = os.path.join(experiment_2_root, "QuPath_output")
experiment_2_dirs = ["41CM", "42CF", "43CM", "44CF", "49CM", "50CF", "51CM", "52CF", "58CF", "60CF", "74CF", "76CF", "81CM", "83CM", "89CM", "91CM", "45SM", "46SF", "47SM", "48SF", "53SM", "54SF", "55SM", "56SF", "62SF", "64SF", "78SF", "80SF", "85SM", "87SM", "93SM", "95SM"]
# empty results:
# - Control_51C

tracer_key = "Num AF647"
marker = "CFos"

In [ ]:
slice_root = f"{experiment_1_animals_root}/{experiment_1_dirs[2]}"
image = "2022_04_22__12_22__2525.czi - Scene #01"
slice = BraiAn.BrainSlice(AllenBrain,
                    f"{slice_root}/results/{image}_regions.txt",
                    f"{slice_root}/regions_to_exclude/{image}_regions_to_exclude.txt",
                    experiment_1_dirs[2], image, experiment_1_area_key, tracer_key, marker)
slice.data

In [ ]:
experiment_1_slices = [BraiAn.SlicedBrain(animal_dir,
                                        os.path.join(experiment_1_animals_root, animal_dir),
                                        AllenBrain,
                                        experiment_1_area_key,
                                        tracer_key,
                                        marker,
                                        area_units="µm2")
                    for animal_dir in experiment_1_dirs]
print(f"NOTE: Imported all brain slices from {str(len(experiment_1_slices))} animals of {experiment_1_name} experiment.")

experiment_2_slices = [BraiAn.SlicedBrain(animal_dir,
                                        os.path.join(experiment_2_animals_root, animal_dir),
                                        AllenBrain,
                                        experiment_2_area_key,
                                        tracer_key,
                                        marker,
                                        area_units="µm2")
                    for animal_dir in experiment_2_dirs]
print(f"NOTE: Imported all brain slices  {str(len(experiment_2_slices))} animals of {experiment_2_name} experiment.")

In [ ]:
experiment_1_sum_brains = [BraiAn.AnimalBrain(sliced_brain) for sliced_brain in experiment_1_slices if len(sliced_brain.slices) > 0]
experiment_2_sum_brains = [BraiAn.AnimalBrain(sliced_brain) for sliced_brain in experiment_2_slices if len(sliced_brain.slices) > 0]

In [ ]:
for brain in experiment_1_sum_brains + experiment_2_sum_brains:
    if "Left: root" not in brain.data.index:
        print(f"{brain.name} has no 'Left: root' area")
    if "Right: root" not in brain.data.index:
        print(f"{brain.name} has no 'Right: root' area")

In [ ]:
experiment_1 = BraiAn.AnimalGroup(experiment_1_name, experiment_1_sum_brains, AllenBrain)
experiment_2 = BraiAn.AnimalGroup(experiment_2_name, experiment_2_sum_brains, AllenBrain)

In [ ]:
normalization = "Density"
threshold = 700 # Only plot bars with value larger than threshold
experiments = [experiment_1, experiment_2]
regions_to_plot = ["root"] #BraiAn.regions_to_plot(groups=groups, normalization=normalization, threshold=threshold)
fig = BraiAn.plot_groups(normalization, AllenBrain, experiment_1, experiment_2,
                            selected_regions=regions_to_plot, use_acronyms=False,
                            width=900, barheight=100)
fig.show()

file_title = f"barplot_{experiment_1.marker}_{normalization}_{len(experiments)}groups.png"